<img src='logo.png' style='display: block;height: 61px;float: left;padding: .75rem 1.25rem;'>

# Prueba 2: Analizado los crímenes en la Ciudad de Nueva York

## Preliminares
___

### Enunciado:

En esta ocasión trabajaremos con datos públicos del departamento de policía de New York. El dataset es  llamado `stop_and_frisk_data`  y  contiene información sobre interrogaciones y detenciones realizadas por el departamento de policia de NY en la vía pública. El diccionario de atributos se encuentra en el archivo [2009 SQF File Spec.xlsx](./2009_SQF_File_Spec.xlsx).

Para todo nuestro estudio utilizaremos los datos correspondientes al año 2009 como conjunto de entrenamiento y los datos del 2010 como conjunto de pruebas. Hay que hacer notar que los datos que estamos utilizando son un muestreo del 1% de la cantidad de registros reales que contiene el dataset, esta decisión fue tomada debido a los largos tiempos de entrenamiento y procesamiento que requiere el volumén de datos reales.

* Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook, archivosauxiliares y csv).

* Una vez terminada la prueba, comprime la carpeta y sube el `.zip` a la seccióncorrespondiente.

### Objetivos

Para alcanzar el objetivo general, su trabajo se puede desagregar en los siguientes puntos:

1. Dado la gran cantidad de atributos, se le entrega un script `preproc_nyc_sqf.py` que normaliza la cantidad de atributos. Haga uso de la función `create_suitable_dataframe` para igualar el benchmark de los atributos.


2. Debe analizar de forma exploratoria los atributos. Reporte la cantidad de datos perdidos y presente su esquema de recodificación.


3. Generar un modelo predictivo que __condicional__ a las características medidas del sospechoso, prediga si un determinado procedimiento concluirá en un arresto o no. Para ello, guíase por los siguentes lineamientos:
  * Entrene por lo menos 1 modelo que sea capaz de predecir si se producirá un arresto o no. Una vez que encuentre un modelo satisfactorio, reporte al menos dos métricas de desempeño.
  * Refine aquellos atributos relevantes. Encuentre por lo menos 30 atributos que explique la importancia relativa y ordénelos por orden de importancia.
  * Finalmente, reporte la probabilidad que un individuo sea arrestado en uno de los cinco barrios, condicional al género y condicional a la raza.


4. Genere al menos cinco modelos predictivos que permitan determinar si el procedimiento policial concluirá en alguna acción violenta.
  * Para ello, debe generar un nuevo atributo como vector objetivo que indique cuándo hubo violencia o no. Éste debe ser creado a partir de atributos existentes que indiquen el tipo de violencia. El detalle de los atributos que se consideran violentos se detalla a continuación:
```python
'pf_hands', 'pf_wall', 'pf_grnd', 'pf_drwep', 'pf_baton', 
'pf_hcuff', 'pf_pepsp', 'pf_other'
```


5. Seleccione los 2 mejores modelos, serialicelos y envíelos a evaluación. Recuerde que el modelo serializado debe ser posterior al `fit` , para poder ejecutar `predict` en los nuevos datos.


6. La evaluación del modelo será realizada en función a un conjunto de datos reservados al cual no tienen acceso.

> Para resolver el problema planteado, "determinar si un procedimiento concluirá en un arresto o no", vamos a usar técnicas y algoritmos de clasificación. Tenemos todos los elementos para resolver un problema de aprendizaje supervisado, ya que tenemos la variable dependiente __arstmade__ (se realizó la una detención) y se plantea como un atributo discreto, en este caso binario. Además disponemos de una base de datos con suficientes muestras. Para entrenar el modelo usaremos los datos del año 2009.
>
> Inicialmente nos vemos tentados a usar una solución desde la econometría, no obstante las herramientas disponibles para machine learning nos permitiran navegar sobre soluciones y encontrar un modelo de la solución balanceado entre el sesgo y la varianza. Dada la cantidad de atributos nominales y la bondad evaluada en los metodos de __ensambles__ vamos a apostar por obtener buenos resultados con __Random Forest__, __Adaptive Boosting__ y __Gradient Boosting__, no obstante y para efectos de comparacón de modelos, vamos a implementar el algoritmo __SVC__ con Kernel RBF y __LDA__ (análisis lineal discriminante) o __QDA__ (análisis cuadrático discriminante).
>
> Para __validar la solución__ extraeremos medidas desde la matriz de confusión, revisaremos __accuracy__, __Precision__ y __Recall__, luego revisaremos la media armónica entre Precision y Recall (__F1__). Para terminar usaremos el despliegue de la curva __ROC__ para justificar la bondad del modelo. Es decir, explicar que % adicional explica el modelo por sobre una asignación aleatorea (0.5). Para validar el modelo usaremos datos del __2010__, así revisamos la capacidad de predicción empírica del modelo.

### Vector objetivo:
>
> Para el objetivo 3 nuestro vector objetivo arstmade (se realizó o no la detención).
>
> Para el objetivo 4 nuestro vector objetivo es una nueva variable que determinará si el procedimiento policial cloncluirá en una acción violenta.

### Tipo de problema:
>
> Estamos frente a un problema de clasifición con información previa sobre las clases y atributos de un procedimiento policial. En este sentido, tenemos la data para abordar la solución con herramientas de clasificación supervisada.

### Modelos:

> Los hiperparámetros de los modelos serán calibrados mediante búsqueda de grilla con validaciones cruzadas (GridSeachCV), en la medida que se requiera. 
>
> __Gradient Boosting__: 
>  + param_grid={'learning_rate': [0.01, 0.1, 0.5], 'n_estimators': [50, 100, 500, 1000, 2000]}
>  + 5 Validaciones Cruzadas
>
> __Adaptive Boosting__: 
>  + param_grid={'learning_rate': [0.01, 0.1, 0.5],'n_estimators': [50, 100, 500, 1000, 2000], 'subsample': [0.1,0.5,0.9]}
>  + 5 Validaciones Cruzadas
>
> __Random Forest__: 
>  + n_estimators = range(20, 1000, 50)
>  + max_features="log2"
>  + oob_score = True
>
> __SVC__ con Kernel __RBF__:
>  + param_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000], 'gamma': [0.0000001, 0.0001, 0.001, 0.01, 0.1, 1, 10]}
>  + 5 Validaciones Cruzadas
>
> __QDA__ (análisis cuadrático discriminante): Es probable que dejemos iniciado el hiperparámetro priors, con la probabilidad a priori del 2009.
>
> __BernoulliNB__ (para objetivo 3) o __MultinomialNB__ (para objetivo 4): Sin modificar hiperparámetros.

### Métricas:
> * Accuracy
> * Precision
> * Recall
> * fi_score
> * ROC

### Preprocesamiento
>
> Para el caso de los atributos categoricos se binarizarán y en caso de que los atributos continuos estén desbalanceados procederemos a recodificarlos con su logaritmo. Adicionalmente usaremos la función `create_suitable_dataframe` para igualar el benchmark de los atributos.

## Aspectos computacionales
___

> Se utilizan librerías bases para el tratamiento de datos y algunos aspectos básicos de cálculo y gráficos.

In [1]:
# Se importa libreria para el manejo de bases de datos
import pandas as pd
# Se importa libreria para el manejo de operaciones de cálculo
import numpy as np
# Se importa libreria para el manejo de gráficos
import matplotlib.pyplot as plt
# Se importa libreria para manejo de funciones estadisticas y probabilidades
import seaborn as sns
# Librería para visualizar patrones de datos perdidos
import missingno as msngo
# Se importa libreria para el menejo de warning
import warnings
# Se importa libreria para el trabajo de expresiones reguilares
import re

> Definimos algunos aspectos de ambiente y valores por defecto de visualización.

In [2]:
# Por defecto, matplotlib crea una figura en una ventana separada.
# podemos hacer que las figuras aparezcan en línea dentro del notebook; lo hacemos ejecutando:
%matplotlib inline
# Se ignoran los warning para evitar suciedad en la ejecución
warnings.filterwarnings(action='ignore')
# Se define el estilo de gráficos a usar
plt.style.use('seaborn-pastel')
# Se define el tamaño de los paños de los gráficos por defecto
plt.rcParams['figure.figsize'] = (14, 8)
# Dado que vamos a supervisar datos no limitaremos la cantidad de columnas a mostrar en el despliegue del dataframe
pd.options.display.max_columns = None

> Se utilizan librerías de __sklear__ para resolver el problema desde el enfoque de machine learning

In [3]:
# Método de ensamble secuencial, corrige mediante la función de pérdida usando el descenso por el gradiente.
from sklearn.ensemble import GradientBoostingClassifier
# Método de ensamble secuencial, corrige por las tasas de clasificaciones incorrectas
from sklearn.ensemble import AdaBoostClassifier
# Método de ensamble paralelo, aplica bootstrapping por fila y columnas. 
from sklearn.ensemble import RandomForestClassifier
# Método de clasificación mediante la maximización de margen de clases.
from sklearn.svm import SVC
# Método de Análisis cuadrático discriminante, los atributos tienen una forma cuadrátio.
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# Método que implementa Bayes ingenuo para el caso dos clases y multiclase.
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
# Métricas para evaluar modelos de clasificación
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
# Métodos de selección de muestras
from sklearn.model_selection import GridSearchCV, train_test_split
# Metodo para el procesamiento de los datos en una tuberias
from sklearn.pipeline import Pipeline

> Se utiliza la librería __pickle__ que contiene la función dump, que permite guardar el modelo desarrollado. 

In [4]:
# Método para hacer dump del modelo y objetos
import pickle

> Se utiliza <strong>librería propia</strong> que continen funciones auxiliares, necesarias para el desarrollo de desafíos.

In [5]:
# Se importa libreria con funciones auxiliares
import ancilliary_funcs as afx

> Se utiliza <strong>librería helpers.py</strong> que continen funciones auxiliares, necesarias para el desarrollo de desafíos.

In [6]:
import helpers as hlp

> Se utiliza __librería preproc_nyc_sqf.py__ provista para abordar temas de procesamiento de atributos.

In [7]:
import preproc_nyc_sqf